In [45]:
# Initialize python libs & SQL creads
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import pandas as pd
from pandasql import sqldf
import streamlit as st
import pull_nrg_data
import json
import http.client
import certifi
import ssl
import os
import timeit
import pickle
import altair as alt
import pytz

In [24]:
def release_token(accessToken):
    server = 'api.nrgstream.com'
    path = '/api/ReleaseToken'
    headers = {'Authorization': f'Bearer {accessToken}'}
    context = ssl.create_default_context(cafile=certifi.where())
    conn = http.client.HTTPSConnection(server,context=context)
    conn.request('DELETE', path, None, headers)
    res = conn.getresponse()
    print('token released')

with open('./default_pickle.pickle', 'rb') as handle:
    default_pickle = pickle.load(handle)
print(default_pickle['accessToken'])
#release_token(default_pickle['accessToken'])

In [33]:
t = datetime(2022,1,1).date()
default_pickle = {
    'accessToken':'',
    'current_data':'',
    'daily_outage_dfs':[0]*7,
    'monthly_outage_dfs':[0]*7,
    'alert_dates':{
        'Biomass & Other':t, 
        'Coal':t,
        'Dual Fuel':t,
        'Energy Storage':t,
        'Hydro':t,
        'Natural Gas':t,
        'Solar':t,
        'Wind':t,
        'Intertie':t
    }
}
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)
# Add default current values to dict
with open('./current_df.pickle', 'rb') as handle:
    current_df = pickle.load(handle)
default_pickle['current_data'] = current_df
# Add default monthly values to dict
with open('./monthly_df.pickle', 'rb') as handle:
    monthly = pickle.load(handle)
for i in range(7):
    default_pickle['monthly_outage_dfs'][i] = (datetime.today().date(),monthly)
# Add default daily values to dict
with open('./daily_df.pickle', 'rb') as handle:
    daily = pickle.load(handle)
for i in range(7):
    default_pickle['daily_outage_dfs'][i] = (datetime.today().date(),daily)
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:

with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [30]:
with open('./default_pickle.pickle', 'rb') as handle:
    default_pickle = pickle.load(handle)
default_pickle

{'accessToken': '',
 'current_data': (datetime.datetime(2022, 9, 7, 11, 36, 49, 310651),
                timeStamp  value      fuelType
  0   2022-09-07 00:00:00   1256          Coal
  1   2022-09-07 00:05:00   1260          Coal
  2   2022-09-07 00:10:00   1256          Coal
  3   2022-09-07 00:15:00   1257          Coal
  4   2022-09-07 00:20:00   1261          Coal
  ..                  ...    ...           ...
  135 2022-09-07 11:15:00    -59  Saskatchewan
  136 2022-09-07 11:20:00    -59  Saskatchewan
  137 2022-09-07 11:25:00    -59  Saskatchewan
  138 2022-09-07 11:30:00    -58  Saskatchewan
  139 2022-09-07 11:35:00    -58  Saskatchewan
  
  [1540 rows x 3 columns]),
 'daily_outage_dfs': [(datetime.date(2022, 9, 7),
        timeStamp  value  fuelType
   0   2022-09-02    290  Intertie
   1   2022-09-03      0  Intertie
   2   2022-09-04      0  Intertie
   3   2022-09-05      0  Intertie
   4   2022-09-06      0  Intertie
   ..         ...    ...       ...
   115 2022-12-26    

In [69]:
with open('./monthly_df.pickle', 'rb') as handle:
    monthly_outage = pickle.load(handle)


In [68]:

a = daily_outage[daily_outage['timeStamp'].dt.date<((now+timedelta(days=90)).date())]

In [76]:
now = datetime.now(pytz.timezone('America/Edmonton'))
monthly_outage[monthly_outage['timeStamp'].dt.date<(now.date()+timedelta(days=90))]

,timeStamp,value,fuelType
9,2022-10-01,470,Coal
10,2022-11-01,30,Coal
11,2022-12-01,0,Coal
9,2022-10-01,1130,Natural Gas
10,2022-11-01,740,Natural Gas
11,2022-12-01,460,Natural Gas
9,2022-10-01,0,Dual Fuel
10,2022-11-01,0,Dual Fuel
11,2022-12-01,0,Dual Fuel
9,2022-10-01,90,Hydro
